In [1]:
!git clone https://github.com/QwenLM/Qwen.git

Cloning into 'Qwen'...
remote: Enumerating objects: 1753, done.
remote: Counting objects: 100% (842/842), done.
remote: Compressing objects: 100% (321/321), done.
remote: Total 1753 (delta 635), reused 605 (delta 506), pack-reused 911 (from 1)
Receiving objects: 100% (1753/1753), 35.93 MiB | 12.85 MiB/s, done.
Resolving deltas: 100% (1034/1034), done.


In [2]:
%cd Qwen

/content/Qwen


In [3]:
!pip install deepspeed transformers==4.32.0 peft pydantic==1.10.13 transformers_stream_generator einops tiktoken optimum auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of deepspeed to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 69.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 

In [7]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.8/464.8 kB 24.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.0-cp310-cp310-linux_x86_64.whl size=4266269 sha256=fdc6987416796be890b5067dfa3db0a9bcab054031a601a1846d54d9472d126e
  Stored in directory: /root/.cache/pip/wheels/96/17/12/83db63ee0ae5c4b040ee87f2e5c813aea4728b55ec6a37317c
Successfully built mpi4py


In [5]:
import pandas as pd
import json

df = pd.read_json('https://raw.githubusercontent.com/wangyuxinwhy/uniem/main/examples/example_data/riddle.jsonl', lines=True)
df = df.rename(columns={'instruction': 'user', 'output': 'assistant'})

chat_input = df["user"].tolist()
chat_output = df["assistant"].tolist()

all_chat = []
for i in range(len(chat_input)):
    user = str(chat_input[i])
    assistant = str(chat_output[i])
    conversations = [{"from": "user", "value": user}, {"from": "assistant", "value": assistant}]
    chat_id = "identity_" + str(i)
    chat = {"id": chat_id, "conversations": conversations}
    all_chat.append(chat)

file_path = 'chat.json'

with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(all_chat, json_file, ensure_ascii=False)
all_chat[0]

{'id': 'identity_0',
 'conversations': [{'from': 'user',
   'value': '猜谜语：一身卷卷细毛，吃的青青野草，过了数九寒冬，无私献出白毛。 （打一动物）'},
  {'from': 'assistant', 'value': '谜底：白羊'}]}

In [8]:
all_chat[0]

{'id': 'identity_0',
 'conversations': [{'from': 'user',
   'value': '猜谜语：一身卷卷细毛，吃的青青野草，过了数九寒冬，无私献出白毛。 （打一动物）'},
  {'from': 'assistant', 'value': '谜底：白羊'}]}

In [9]:
!git clone https://modelscope.cn/qwen/Qwen-1_8B-Chat-Int4.git

Cloning into 'Qwen-1_8B-Chat-Int4'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 119 (delta 39), reused 112 (delta 32), pack-reused 0
Receiving objects: 100% (119/119), 16.18 MiB | 13.38 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [10]:
!export CUDA_DEVICE_MAX_CONNECTIONS=1

!export CUDA_VISIBLE_DEVICES=0

!python finetune.py \
  --model_name_or_path Qwen-1_8B-Chat-Int4 \
  --data_path chat.json \
  --fp16 True \
  --output_dir output_qwen \
  --num_train_epochs 1 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 1 \
  --gradient_accumulation_steps 8 \
  --evaluation_strategy "no" \
  --save_strategy "steps" \
  --save_steps 1000 \
  --save_total_limit 10 \
  --learning_rate 3e-4 \
  --weight_decay 0.1 \
  --adam_beta2 0.95 \
  --warmup_ratio 0.01 \
  --lr_scheduler_type "cosine" \
  --logging_steps 1 \
  --report_to "none" \
  --model_max_length 512 \
  --lazy_preprocess True \
  --gradient_checkpointing \
  --use_lora \
  --q_lora \
  --deepspeed finetune/ds_config_zero2.json


[2024-09-23 18:23:05,846] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
/usr/local/lib/python3.10/dist-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/usr/local/lib/python3.10/dist-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/usr/local/lib/python3.10/dist-packages/transformers/uti

In [14]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

model = AutoPeftModelForCausalLM.from_pretrained("output_qwen", device_map="auto", trust_remote_code=True).eval()
tokenizer = AutoTokenizer.from_pretrained("output_qwen", trust_remote_code=True)
response, history = model.chat(tokenizer, "猜谜语：一身卷卷细毛，吃的青青野草，过了数九寒冬，无私献出白毛。 （打一动物）", history=None)
print(response)

Some weights of the model checkpoint at Qwen-1_8B-Chat-Int4 were not used when initializing QWenLMHeadModel: ['transformer.h.6.mlp.w1.bias', 'transformer.h.20.mlp.w2.bias', 'transformer.h.5.mlp.c_proj.bias', 'transformer.h.7.attn.c_proj.bias', 'transformer.h.23.attn.c_proj.bias', 'transformer.h.23.mlp.c_proj.bias', 'transformer.h.3.attn.c_proj.bias', 'transformer.h.4.mlp.w2.bias', 'transformer.h.14.mlp.w1.bias', 'transformer.h.6.attn.c_proj.bias', 'transformer.h.13.mlp.c_proj.bias', 'transformer.h.3.mlp.c_proj.bias', 'transformer.h.18.mlp.c_proj.bias', 'transformer.h.15.mlp.c_proj.bias', 'transformer.h.15.mlp.w2.bias', 'transformer.h.22.mlp.c_proj.bias', 'transformer.h.11.mlp.w1.bias', 'transformer.h.12.mlp.w2.bias', 'transformer.h.1.mlp.w1.bias', 'transformer.h.16.mlp.c_proj.bias', 'transformer.h.17.attn.c_proj.bias', 'transformer.h.18.attn.c_proj.bias', 'transformer.h.14.mlp.c_proj.bias', 'transformer.h.20.mlp.c_proj.bias', 'transformer.h.14.attn.c_proj.bias', 'transformer.h.3.mlp.w2

谜底：老牛
